# 자연어 처리
# 토픽 모델링 : 잠재 의미 분석(LSA)

토픽 모델링(Topic Modeling)이란 문서 집합에 숨어 있는 '주제'를 찾아내는 텍스트 마이닝기법 중 하나이다. 문서에 함축되어 있는 주요 주제를 효과적으로 찾아낼 수 있다. 

토픽 모델링에 자주 사용되는 기법은 LSA(Latent Semantic Analysis)와 LDA(Latent Dirichlet Allocation)이다. 

기존의 카운트 기반 벡터화, TF-IDF로 얻은 문서-단어 행렬은 단어의 빈도수를 이용하여 행렬을 만들었다. 하지만 단어의 빈도수만으로는 주제를 파악하기 힘들다. 빈도로 단어의 의미를 정확히 고려하지 못하기 때문이다.이에 대한 방안으로 나온 것이 문서의 잠재된 의미를 찾아낸느 잠재 의미 분석(LSA)이다. LSA는 SVD를 활용하여 문서에 주제를 찾아내는 것이기 때문에 SVD를 이해했으면 LSA 이해는 금방이다.

## LSA 실습

In [2]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

In [3]:
dataset = fetch_20newsgroups(shuffle=True,random_state=98, remove=('headers','footers','quotes'))


In [17]:
print('데이터 형태 확인 : ',dataset.keys())

데이터 형태 확인 :  dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [18]:
#fetch_20newsgroups : 20개의 토픽을 가지 11,314개의 뉴스 기사 데이터 셋
documents = dataset.data
len(documents)

11314

In [20]:
documents[0]

'\nSuperficially a good answer, but it isn\'t that simple.  An awful lot of the\nstarvation and poverty in the world is directly caused by the economic\npolicies of the Western countries, as well as by the diet of the typical\nWesterner.  For instance, some third-world countries with terrible\nmalnutrition problems export all the soya they can produce -- so that it can\nbe fed to cattle in the US, to make tender juicy steaks and burgers.  They\nhave to do this to get money to pay the interest on the crippling bank loans\nwe encouraged them to take out.  Fund-raising for Ethiopia is a truly bizarre\nidea; instead, we ought to stop bleeding them for every penny they\'ve got.\n\nPerhaps it\'s more accurate to say that there\'s a Western ethic against\nWestern infanticide.  All the evidence suggests that so long as the children\nare dying in the Third World, we couldn\'t give a shit.  And that goes for the\nsupposed "Pro-Life" movement, too.  They could save far more lives by\nfighting aga

In [22]:
documents[11313]

"Various posts about shafties can't do wheelies:\n\n\nUh, folks, the shaft doesn't have diddleysquatpoop to do with it. I can get\nthe front wheel off the ground on my /5, ferchrissake!"

위와 같은 문서 형태를 가진 문서는 총 11,314개 이다. 문서의 토픽은 target_names에 저장되어 있다.

In [23]:
dataset.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

LSA 전 데이터 전처리 : 알파벳 이외의 문자 제거, 길이가 3 이하인 문자도 제거, 대소문자 구분 없애기

In [25]:
news_df = pd.DataFrame({'document':documents})
# 알파벳 이외 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]"," ")
#길이 3 이하 문자 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 소문자 바꾸기
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [38]:
news_df.head()

,document,clean_doc
0,"\nSuperficially a good answer, but it isn't th...",superficially good answer that simple awful st...
1,"OHIO HOUSE OF REPRESENTATIVE TUEDAY, APRIL 6,...",ohio house representative tueday april represe...
2,\n +------------------------------------------...,kevin marshall operational support motorola ec...
3,Don A.B. Lindbergh meinte am 15.04.93\nzum The...,lindbergh meinte thema diamond mouse cursor an...
4,\n\n\n I HEARTILY agree. Now that the BAT...,heartily agree that batf warrant been unsealed...


doc TF-IDF 벡터화 1,000 단어까지만 벡터화 진행(max_features = 1000)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english',max_features=1000,max_df=0.5,smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape

(11314, 1000)

11314개 문서를 1,000 단어를 활용하여 문서-단어 행렬 생성

In [27]:
from sklearn.decomposition import TruncatedSVD #토픽모델링

svd_model = TruncatedSVD(n_components=20, algorithm = 'randomized', n_iter=100, random_state=12)
svd_model.fit(X)
svd_model.components_.shape

(20, 1000)

뉴스 토픽이 총 20개 이므로 n_components는 20으로 설정, Truncated SVD 분해시 상위 20개 특이값만 사용

In [28]:
svd_model.singular_values_

array([17.15952833,  9.93882749,  8.17139855,  7.92032011,  7.62377374,
        7.5257242 ,  7.25096862,  7.00623237,  6.88289372,  6.85602044,
        6.68476301,  6.56045782,  6.52895929,  6.42222944,  6.33939436,
        6.21686249,  6.17477882,  6.09487639,  6.00247117,  5.90654237])

In [30]:
terms = vectorizer.get_feature_names()
len(terms)

1000

In [34]:
n = 8 #주요 단어 8개 출력
components = svd_model.components_
for index, topic in enumerate(components):
    print('Topic %d: '%(index+1),[terms[i] for i in topic.argsort()[: -n -1:-1]])

Topic 1:  ['just', 'like', 'know', 'people', 'think', 'does', 'good', 'time']
Topic 2:  ['thanks', 'windows', 'card', 'drive', 'mail', 'file', 'advance', 'files']
Topic 3:  ['game', 'team', 'year', 'games', 'drive', 'season', 'good', 'players']
Topic 4:  ['drive', 'scsi', 'disk', 'hard', 'problem', 'drives', 'just', 'card']
Topic 5:  ['drive', 'know', 'thanks', 'does', 'just', 'scsi', 'drives', 'hard']
Topic 6:  ['just', 'like', 'windows', 'know', 'does', 'window', 'file', 'think']
Topic 7:  ['just', 'like', 'mail', 'bike', 'thanks', 'chip', 'space', 'email']
Topic 8:  ['does', 'know', 'chip', 'like', 'card', 'clipper', 'encryption', 'government']
Topic 9:  ['like', 'card', 'sale', 'video', 'offer', 'jesus', 'good', 'price']
Topic 10:  ['like', 'drive', 'file', 'files', 'sounds', 'program', 'window', 'space']
Topic 11:  ['people', 'like', 'thanks', 'card', 'government', 'windows', 'right', 'think']
Topic 12:  ['think', 'good', 'thanks', 'need', 'chip', 'know', 'really', 'bike']
Topic 1

Topic 3을 스포츠 관련 기사일 것이고, Topic 18은 컴퓨터 관련 기사일 것이다. 불용어를 제외하면 더 정확한 결과가 나올 수 있을듯 하다.

In [52]:
dataset.target_names # target과 연결해보기

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']